In [1]:
import urllib.parse
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import RDF, RDFS, FOAF, OWL, XSD, DC, DCTERMS 
import pandas as pd

In [2]:
base_uri = "http://www.sanitasicilia.it/resource/"

g = Graph()

sso = Namespace("http://www.sanitasicilia.it/ontology/")

g.bind("sso", sso)

ssr = Namespace("http://www.sanitasicilia.it/resource/")
g.bind("ssr", ssr)


In [4]:
#Comuni

def urify(ns, testo):
    testo=testo.replace(" ","_").replace("\'","")
    return ns+urllib.parse.quote(testo)

def addTriples(row):
    res = URIRef(urify(base_uri, row[0]))
    g.add([res, URIRef(RDF.type), URIRef(sso.Comune)])
    g.add([res, sso.hasName, Literal(row[0], datatype=XSD.string)])
    g.add([res, sso.hasTotalPopulation, Literal(row[1], datatype=XSD.integer)])
    g.add([res, sso.hasProvince, Literal(row[2], datatype=XSD.string)])
    g.add([res, sso.hasProvinceAcr, Literal(row[3], datatype=XSD.string)])

comuni_df = pd.read_csv("../datasets/ready/comuni_sicilia.csv")

comuni_df.apply(lambda row : addTriples(row), axis=1)

g.serialize(destination='../datasets/rdf/comuni_sicilia.ttl', 
            format='turtle')

<Graph identifier=N7b5761552d66458194fa951b1cc118e7 (<class 'rdflib.graph.Graph'>)>

In [25]:
#Farmacie e Parafarmacie
def addTriples(row):
    res = URIRef(urify(base_uri, row[5]))
    g.add([res, URIRef(RDF.type), URIRef(sso.Farmacia)])
    g.add([res, sso.isIn, URIRef(urify(base_uri, row[2].title()))])
    g.add([res, sso.hasCap, Literal(row[3], datatype=XSD.integer) ])
    g.add([res, sso.hasAddress, Literal(row[4], datatype=XSD.string) ])
    g.add([res, sso.hasName, Literal(row[5], datatype=XSD.string) ])
    g.add([res, sso.hasLatitude, Literal(row[6], datatype=XSD.decimal) ])
    g.add([res, sso.hasLongitude, Literal(row[7], datatype=XSD.decimal) ])
    g.add([res, sso.hasVatNumber, Literal(row[8], datatype=XSD.integer) ])


g = Graph()

farmacie_df = pd.read_csv("../datasets/ready/farmacie.csv", sep=';')
farmacie_df["LATITUDINE"] = farmacie_df["LATITUDINE"].apply(lambda x : float(x.replace(',', '.')) if x != '-' else 0.0)
farmacie_df["LONGITUDINE"] = farmacie_df["LONGITUDINE"].apply(lambda x : float(x.replace(',', '.')) if x != '-' else 0.0)
farmacie_df["PARTITAIVA"] = farmacie_df["PARTITAIVA"].apply(lambda x : x if x != '-' else 0)

parafarmacie_df = pd.read_csv("../datasets/ready/parafarmacie.csv", sep=';')
parafarmacie_df["LATITUDINE"] = parafarmacie_df["LATITUDINE"].apply(lambda x : float(x.replace(',', '.')) if x != '-' else 0.0)
parafarmacie_df["LONGITUDINE"] = parafarmacie_df["LONGITUDINE"].apply(lambda x : float(x.replace(',', '.')) if x != '-' else 0.0)
parafarmacie_df["PARTITAIVA"] = parafarmacie_df["PARTITAIVA"].apply(lambda x : x if x != '-' else 0)


farmacie_df.apply(lambda row : addTriples(row), axis=1)

g.serialize(destination='../datasets/rdf/farmacie.ttl', 
            format='turtle')

g = Graph() #attualmente, per debug rdf separati

parafarmacie_df.apply(lambda row : addTriples(row), axis=1)
g.serialize(destination='../datasets/rdf/parafarmacie.ttl', 
            format='turtle')

<Graph identifier=N7faa269fb3cc4889a050c85f36c71aa3 (<class 'rdflib.graph.Graph'>)>